![Two data scientists working on a dashboard.](hr-image-small.png)

A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

In [2]:
# Import necessary libraries
import pandas as pd

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


In [4]:
# Create a copy of ds_jobs for transforming
ds_jobs_transformed = ds_jobs.copy()

## Columns containing categories with only two factors must be stored as Booleans (bool).

In [5]:
for col in list(ds_jobs_transformed.columns):
    print(ds_jobs_transformed[col].value_counts())
    print('\n--------------------')

# --> relative experience and job_change are the columns with 2 values

student_id
23834    1
8949     1
29725    1
11561    1
33241    1
        ..
29452    1
699      1
27107    1
402      1
28806    1
Name: count, Length: 19158, dtype: int64

--------------------
city
city_103    4355
city_21     2702
city_16     1533
city_114    1336
city_160     845
            ... 
city_111       3
city_129       3
city_121       3
city_140       1
city_171       1
Name: count, Length: 123, dtype: int64

--------------------
city_development_index
0.920    5200
0.624    2702
0.910    1533
0.926    1336
0.698     683
         ... 
0.649       4
0.807       4
0.781       3
0.625       3
0.664       1
Name: count, Length: 93, dtype: int64

--------------------
gender
Male      13221
Female     1238
Other       191
Name: count, dtype: int64

--------------------
relevant_experience
Has relevant experience    13792
No relevant experience      5366
Name: count, dtype: int64

--------------------
enrolled_university
no_enrollment       13817
Full time course     3757
Part t

In [6]:
ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].map({'Has relevant experience':1 , 'No relevant experience':0})

ds_jobs_transformed['relevant_experience'] = ds_jobs_transformed['relevant_experience'].astype('bool')
ds_jobs_transformed['job_change'] = ds_jobs_transformed['job_change'].astype('bool')

ds_jobs_transformed.dtypes

student_id                  int64
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
job_change                   bool
dtype: object

## Columns containing integers only must be stored as 32-bit integers (int32).

In [7]:
ds_jobs_transformed['student_id'] = ds_jobs_transformed['student_id'].astype('int32')
ds_jobs_transformed['training_hours'] = ds_jobs_transformed['training_hours'].astype('int32')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float64
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

## Columns containing floats must be stored as 16-bit floats (float16).

In [8]:
ds_jobs_transformed['city_development_index'] = ds_jobs_transformed['city_development_index'].astype('float16')

ds_jobs_transformed.dtypes

student_id                  int32
city                       object
city_development_index    float16
gender                     object
relevant_experience          bool
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int32
job_change                   bool
dtype: object

## Columns containing nominal categorical data must be stored as the category data type

In [9]:
nominal_columns  = ['city', 'gender', 'major_discipline', 'company_type']

for col in nominal_columns :
    ds_jobs_transformed[col] = ds_jobs_transformed[col].astype('category')

ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university         object
education_level             object
major_discipline          category
experience                  object
company_size                object
company_type              category
last_new_job                object
training_hours               int32
job_change                    bool
dtype: object

## Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.

In [10]:
ordinal_columns = ['enrolled_university', 'education_level', 'experience', 'company_size',  'last_new_job']

for col in ordinal_columns :
    ds_jobs_transformed[col] = pd.Categorical(ds_jobs_transformed[col], ordered=True)

ds_jobs_transformed['enrolled_university']= ds_jobs_transformed['enrolled_university'].cat.reorder_categories(['no_enrollment', 'Part time course', 'Full time course'], ordered=True)
ds_jobs_transformed['education_level']= ds_jobs_transformed['education_level'].cat.reorder_categories(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
ds_jobs_transformed['experience']= ds_jobs_transformed['experience'].cat.reorder_categories(['<1', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '>20'], ordered=True)
ds_jobs_transformed['company_size']= ds_jobs_transformed['company_size'].cat.reorder_categories(['<10', '10-49',  '50-99', '100-499', '500-999', '1000-4999', '5000-9999' ,'10000+'], ordered=True)
ds_jobs_transformed['last_new_job']= ds_jobs_transformed['last_new_job'].cat.reorder_categories(['never', '1', '2', '3', '4', '>4'], ordered=True)

ds_jobs_transformed.dtypes

student_id                   int32
city                      category
city_development_index     float16
gender                    category
relevant_experience           bool
enrolled_university       category
education_level           category
major_discipline          category
experience                category
company_size              category
company_type              category
last_new_job              category
training_hours               int32
job_change                    bool
dtype: object

In [11]:
ds_jobs_transformed['last_new_job'].dtype

CategoricalDtype(categories=['never', '1', '2', '3', '4', '>4'], ordered=True, categories_dtype=object)

## The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [12]:
filter1= ds_jobs_transformed['experience']>='10'
filter2= ds_jobs_transformed['company_size'] >= '1000-4999'
ds_jobs_transformed = ds_jobs_transformed.loc[filter1&filter2].reset_index(drop=True)

In [13]:
ds_jobs_transformed.head()

c:\Users\El-Wattaneya\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\io\formats\format.py:1458: RuntimeWarning: overflow encountered in cast
  has_large_values = (abs_vals > 1e6).any()


,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,699,city_103,0.919922,NaN,True,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,False
1,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,False
2,22293,city_103,0.919922,Male,True,Part time course,Graduate,STEM,19,5000-9999,Pvt Ltd,>4,141,False
3,26494,city_16,0.910156,Male,True,no_enrollment,Graduate,Business Degree,12,5000-9999,Pvt Ltd,3,145,False
4,2547,city_114,0.925781,Female,True,Full time course,Masters,STEM,16,1000-4999,Public Sector,2,14,False


In [14]:
print(ds_jobs.memory_usage(), '\n---------------')
print(ds_jobs_transformed.memory_usage())



Index                        132
student_id                153264
city                      153264
city_development_index    153264
gender                    153264
relevant_experience       153264
enrolled_university       153264
education_level           153264
major_discipline          153264
experience                153264
company_size              153264
company_type              153264
last_new_job              153264
training_hours            153264
job_change                153264
dtype: int64 
---------------
Index                      132
student_id                8804
city                      7353
city_development_index    4402
gender                    2333
relevant_experience       2201
enrolled_university       2333
education_level           2413
major_discipline          2421
experience                2933
company_size              2565
company_type              2421
last_new_job              2421
training_hours            8804
job_change                2201
dtype: int